In [1]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
np.set_printoptions(precision=3,suppress=True)
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
importlib.reload(tn)

import beck.beck_game
importlib.reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
from beck.beck_players import HumanBeckPlayer
import arena
importlib.reload(arena)
from arena import Arena

import mcts
from bfts import BFTS

import cog_related
importlib.reload(cog_related)
from cog_related import cog_value_net as cvn


importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

from pickle import Pickler, Unpickler
import pickle

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

from utils import *
mctsargs = dotdict({
        'numMCTSSims': 100,
        'cpuct': 2,
        })
import supervised_learning as sl
from importlib import reload
reload(sl)
import keras.backend as K


In [2]:
# checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_6_checkpoints_mcts100_cpuct2_id_1_checkpoint_55'
# file = 'final'
tournament=12
n_res=3
cp = 21

dir_alpha =0
epsilon=0

def get_player(**kwargs):
    tournament = kwargs['tournament']
    n_res = kwargs['n_res']
    cp = kwargs['cp']
    dir_alpha = 0 if 'dir_alpha' not in kwargs.keys() else kwargs['dir_alpha']
    epsilon = 0.25
    cpuct = 2 if 'cpuct' not in kwargs.keys() else kwargs['cpuct']
    mctsargs.cpuct = cpuct
    id = 0 if 'id' not in kwargs.keys() else kwargs['id']
    if tournament=='h':
        human_p = HumanBeckPlayer(game)
        p_human = lambda x:human_p.play(x)
        return None,None,p_human
    else:
#         checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament}/checkpoints_mcts100_cpuct2_id_res{n_res}-0'
        checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament}/checkpoints_mcts100_cpuct{int(cpuct)}_id_res{n_res}-{id}'
        if not os.path.exists(checkpoint_dir):
            print(checkpoint_dir)
            checkpoint_dir = f'/scratch/zz737/fiar/tournaments/tournament_{tournament}/checkpoints_mcts100_cpuct{cpuct:.0e}_id_res{n_res}-{id}'

        file =cp if cp=='best' else f'checkpoint_{cp}'

        # checkpoint_dir = '/scratch/zz737/fiar/sl/resnet/res15_color_Ex_tournament_8_checkpoints_mcts100_cpuct2_id_res3-0_checkpoint_61'
        # file = 'final'

        args = pickle.load(open(os.path.join(checkpoint_dir,'args.p'),'rb'))
        on_c = sl.OthelloNNet_resnet(game, args)
        if tournament==8 or tournament==15:
            nnet_c = sl.NNetWrapper(game,nnet=on_c,args=args)
            nnet_c.load_checkpoint(checkpoint_dir, file+'.pth.tar')
            mcts_c = mcts.MCTS(game, nnet_c,mctsargs) 
            ai_c = lambda x:np.random.choice(np.arange(game.getActionSize()),p=mcts_c.getActionProb(x,temp=1/10, dir_alpha=dir_alpha,epsilon=epsilon))

        else:

            nnet_c = sl.NNetWrapper_color(game,nnet=on_c,args=args)
            nnet_c.load_checkpoint(checkpoint_dir, file+'.pth.tar')
            mcts_c = mcts.MCTS_color(game, nnet_c,mctsargs) 
            ai_c = lambda x,c:np.random.choice(np.arange(game.getActionSize()),p=mcts_c.getActionProb(x,c,temp=1/10,dir_alpha=dir_alpha,epsilon=epsilon))

        return nnet_c, mcts_c, ai_c



In [46]:
mctsargs.cpuct=0.5
# dir_alpha =0.8
# epsilon=0.25

K.clear_session()
# agents = [(8,3,'best'),(8,9,'best'),(12,3,41),(12,9,35),(13,3,35),(13,9,29),(14,3,28),(14,9,20),(15,3,25),(15,9,21),('h',None,None)]
agents = [(8,3,'best'),(8,9,'best'),(12,3,41),(12,9,35),(13,3,35),(13,9,29),(14,3,28),(14,9,20),(15,3,25),(15,9,21)]
# agents = [(8,3,22),(8,9,'best'),(12,3,24),(12,9,21),(13,3,25),(13,9,18)]
# nnet_d = {}
# mcts_d = {}
# ai_d = {}
# for key in agents:
#     nnet_d[key], mcts_d[key], ai_d[key] = get_player(*key)

In [3]:
def find_curr_best_agents(tournament_ind_l = [12,13,14,15,16]):
    agents = []
    for t_ind in tournament_ind_l:
        to_check_dir = f'/scratch/zz737/fiar/tournaments/tournament_{t_ind}'
        for n_res in [3,9]:
            name = f'checkpoints_mcts100_cpuct2_id_res{n_res}-0'
            try:
                cur_best_iter = 1
                for fn in os.listdir(os.path.join(to_check_dir,name)):
                    if 'checkpoint' in fn and 'index' in fn:
                        chpt_iter = fn.split('.')[0] 
                        iter = int(chpt_iter.split('_')[1])
                        if iter > cur_best_iter:
                            cur_best_iter = iter
                agents.append((t_ind,n_res,cur_best_iter,2,0,0))
            except:
                pass
    return agents

In [5]:
agents = find_curr_best_agents()
agents.extend([(8,3,'best',2,0,0),(8,9,'best',2,0,0),(8,3,53,2,0,1),(8,3,49,0.5,0,0)])

# agents  = [(8,3,'best',2,0,0),(8,9,'best',2,0,0),(12,3,55,2,0,0),(12,9,41,2,0,0),(13,3,55,2,0,0),(13,9,39,2,0,0),(14,3,40,2,0,0),(14,9,28,2,0,0),(15,3,35,2,0,0),(15,9,30,2,0,0),(16,3,32,2,0,0),(8,3,18,2,0,1),(8,3,17,0.5,0,0)]
# cpuct_l = [2]
# dir_alpha_l = [0]

import itertools
# all_agents = list(itertools.product(agents,cpuct_l,dir_alpha_l))
keys = ['tournament','n_res','cp','cpuct','dir_alpha','id']
# all_agents = [[*tup[0],*tup[1:]] for tup in all_agents]
all_agents = agents
players = pd.DataFrame(all_agents,columns=keys)
# players.to_pickle('t8_cpuct_diralpha_tournament_players.p')
tour_name = 'new_t_default_updated_3'
players.to_pickle(f'{tour_name}_players.p')

tour_res = pd.DataFrame(index=players.index,columns=players.index)

# tour_res.to_pickle('t8_cpuct_diralpha_tournament_res.p')
tour_res_fn = f'{tour_name}_res.p'
tour_res.to_pickle(tour_res_fn)


In [6]:
import tqdm
from tqdm.notebook import tqdm

In [61]:
# players.loc[11,'n_res'] = 3
# players.loc[12,'n_res'] = 3
players.loc[11,'cp'] = 42
players.loc[12,'cp'] = 41
print(players)
players.to_pickle(f'{tour_name}_players.p')

    tournament  n_res    cp  cpuct  dir_alpha  id
0           12      3    75    2.0          0   0
1           12      9    58    2.0          0   0
2           13      3    75    2.0          0   0
3           13      9    54    2.0          0   0
4           14      3    53    2.0          0   0
5           14      9    44    2.0          0   0
6           15      3    51    2.0          0   0
7           15      9    43    2.0          0   0
8           16      3    55    2.0          0   0
9            8      3  best    2.0          0   0
10           8      9  best    2.0          0   0
11           8      3    42    2.0          0   1
12           8      3    41    0.5          0   0


In [7]:
for id_1,p_dict_1 in tqdm(players.iterrows()):
    p_dict_1 = dict(p_dict_1)
    val_1,tree_1,ai_1 = get_player(**p_dict_1)
    track_color_1 = p_dict_1['tournament'] not in (8,15,'h')
    for id_2,p_dict_2 in tqdm(players.iterrows()):
        if np.isnan(tour_res.loc[id_1,id_2]):
            p_dict_2 = dict(p_dict_2)
            val_2,tree_2,ai_2 = get_player(**p_dict_2)
            track_color_2 = p_dict_2['tournament'] not in (8,15,'h')
    #         print(f'{p_dict_1} vs {p_dict_2}')

            arena = Arena(ai_1,ai_2, game, display=game.display,tree1=tree_1,tree2=tree_2,track_color=[track_color_1, track_color_2], flip_color=False)
            res=arena.playGame(verbose=False,initboard=None)
    #         print(f'{p_dict_1} vs {p_dict_2}: {res}')
            print(f'{list(p_dict_1.values())[:3]} vs {list(p_dict_2.values())[:3]}: {res}')
            tour_res.loc[id_1,id_2] = res
    #         tour_res.to_pickle('new_t_default_res_updated.p')
            tour_res.to_pickle(tour_res_fn)
            K.clear_session()



[12, 3, 94] vs [12, 3, 94]: 0.0001
[12, 3, 94] vs [12, 9, 64]: 1
[12, 3, 94] vs [13, 3, 90]: 0.0001
[12, 3, 94] vs [13, 9, 65]: 0.0001
[12, 3, 94] vs [14, 3, 73]: 1
[12, 3, 94] vs [14, 9, 51]: 1
[12, 3, 94] vs [15, 3, 63]: 0.0001
[12, 3, 94] vs [15, 9, 54]: 1
[12, 3, 94] vs [16, 3, 68]: -1
[12, 3, 94] vs [8, 3, 'best']: 1
[12, 3, 94] vs [8, 9, 'best']: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[12, 3, 94] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[12, 3, 94] vs [8, 3, 49]: 1



[12, 9, 64] vs [12, 3, 94]: 0.0001
[12, 9, 64] vs [12, 9, 64]: 0.0001
[12, 9, 64] vs [13, 3, 90]: 0.0001
[12, 9, 64] vs [13, 9, 65]: 1
[12, 9, 64] vs [14, 3, 73]: 0.0001
[12, 9, 64] vs [14, 9, 51]: 0.0001
[12, 9, 64] vs [15, 3, 63]: 0.0001
[12, 9, 64] vs [15, 9, 54]: 1
[12, 9, 64] vs [16, 3, 68]: 1
[12, 9, 64] vs [8, 3, 'best']: 1
[12, 9, 64] vs [8, 9, 'best']: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[12, 9, 64] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[12, 9, 64] vs [8, 3, 49]: 0.0001



[13, 3, 90] vs [12, 3, 94]: 0.0001
[13, 3, 90] vs [12, 9, 64]: 0.0001
[13, 3, 90] vs [13, 3, 90]: 0.0001
[13, 3, 90] vs [13, 9, 65]: 1
[13, 3, 90] vs [14, 3, 73]: 1
[13, 3, 90] vs [14, 9, 51]: 1
[13, 3, 90] vs [15, 3, 63]: 0.0001
[13, 3, 90] vs [15, 9, 54]: 1
[13, 3, 90] vs [16, 3, 68]: 1
[13, 3, 90] vs [8, 3, 'best']: 1
[13, 3, 90] vs [8, 9, 'best']: -1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[13, 3, 90] vs [8, 3, 53]: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[13, 3, 90] vs [8, 3, 49]: 1



[13, 9, 65] vs [12, 3, 94]: -1
[13, 9, 65] vs [12, 9, 64]: 0.0001
[13, 9, 65] vs [13, 3, 90]: 0.0001
[13, 9, 65] vs [13, 9, 65]: 0.0001
[13, 9, 65] vs [14, 3, 73]: 0.0001
[13, 9, 65] vs [14, 9, 51]: 0.0001
[13, 9, 65] vs [15, 3, 63]: 0.0001
[13, 9, 65] vs [15, 9, 54]: 1
[13, 9, 65] vs [16, 3, 68]: 0.0001
[13, 9, 65] vs [8, 3, 'best']: 1
[13, 9, 65] vs [8, 9, 'best']: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[13, 9, 65] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[13, 9, 65] vs [8, 3, 49]: 1



[14, 3, 73] vs [12, 3, 94]: 0.0001
[14, 3, 73] vs [12, 9, 64]: 0.0001
[14, 3, 73] vs [13, 3, 90]: 1
[14, 3, 73] vs [13, 9, 65]: 1
[14, 3, 73] vs [14, 3, 73]: 0.0001
[14, 3, 73] vs [14, 9, 51]: 1
[14, 3, 73] vs [15, 3, 63]: 0.0001
[14, 3, 73] vs [15, 9, 54]: 1
[14, 3, 73] vs [16, 3, 68]: -1
[14, 3, 73] vs [8, 3, 'best']: 0.0001
[14, 3, 73] vs [8, 9, 'best']: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[14, 3, 73] vs [8, 3, 53]: -1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[14, 3, 73] vs [8, 3, 49]: 1



...

[16, 3, 68] vs [12, 3, 94]: 1
[16, 3, 68] vs [12, 9, 64]: 0.0001
[16, 3, 68] vs [13, 3, 90]: 0.0001
[16, 3, 68] vs [13, 9, 65]: 0.0001
[16, 3, 68] vs [14, 3, 73]: 0.0001
[16, 3, 68] vs [14, 9, 51]: 1
[16, 3, 68] vs [15, 3, 63]: -1
[16, 3, 68] vs [15, 9, 54]: 0.0001
[16, 3, 68] vs [16, 3, 68]: 0.0001
[16, 3, 68] vs [8, 3, 'best']: 0.0001
[16, 3, 68] vs [8, 9, 'best']: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[16, 3, 68] vs [8, 3, 53]: -1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[16, 3, 68] vs [8, 3, 49]: 1



[8, 3, 'best'] vs [12, 3, 94]: 0.0001
[8, 3, 'best'] vs [12, 9, 64]: 1
[8, 3, 'best'] vs [13, 3, 90]: 1
[8, 3, 'best'] vs [13, 9, 65]: 1
[8, 3, 'best'] vs [14, 3, 73]: 0.0001
[8, 3, 'best'] vs [14, 9, 51]: 1
[8, 3, 'best'] vs [15, 3, 63]: 1
[8, 3, 'best'] vs [15, 9, 54]: 1
[8, 3, 'best'] vs [16, 3, 68]: 0.0001
[8, 3, 'best'] vs [8, 3, 'best']: 0.0001
[8, 3, 'best'] vs [8, 9, 'best']: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[8, 3, 'best'] vs [8, 3, 53]: 1
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[8, 3, 'best'] vs [8, 3, 49]: 1



[8, 9, 'best'] vs [12, 3, 94]: -1
[8, 9, 'best'] vs [12, 9, 64]: 0.0001
[8, 9, 'best'] vs [13, 3, 90]: 0.0001
[8, 9, 'best'] vs [13, 9, 65]: 0.0001
[8, 9, 'best'] vs [14, 3, 73]: 0.0001
[8, 9, 'best'] vs [14, 9, 51]: 1
[8, 9, 'best'] vs [15, 3, 63]: 0.0001
[8, 9, 'best'] vs [15, 9, 54]: 1
[8, 9, 'best'] vs [16, 3, 68]: 0.0001
[8, 9, 'best'] vs [8, 3, 'best']: 1
[8, 9, 'best'] vs [8, 9, 'best']: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[8, 9, 'best'] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[8, 9, 'best'] vs [8, 3, 49]: 1

/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1


[8, 3, 53] vs [12, 3, 94]: 1
[8, 3, 53] vs [12, 9, 64]: 1
[8, 3, 53] vs [13, 3, 90]: 1
[8, 3, 53] vs [13, 9, 65]: 1
[8, 3, 53] vs [14, 3, 73]: 0.0001
[8, 3, 53] vs [14, 9, 51]: -1
[8, 3, 53] vs [15, 3, 63]: 0.0001
[8, 3, 53] vs [15, 9, 54]: 0.0001
[8, 3, 53] vs [16, 3, 68]: 1
[8, 3, 53] vs [8, 3, 'best']: 0.0001
[8, 3, 53] vs [8, 9, 'best']: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[8, 3, 53] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[8, 3, 53] vs [8, 3, 49]: 1

/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0


[8, 3, 49] vs [12, 3, 94]: 0.0001
[8, 3, 49] vs [12, 9, 64]: 0.0001
[8, 3, 49] vs [13, 3, 90]: 1
[8, 3, 49] vs [13, 9, 65]: -1
[8, 3, 49] vs [14, 3, 73]: 1
[8, 3, 49] vs [14, 9, 51]: 0.0001
[8, 3, 49] vs [15, 3, 63]: 1
[8, 3, 49] vs [15, 9, 54]: 0.0001
[8, 3, 49] vs [16, 3, 68]: 0.0001
[8, 3, 49] vs [8, 3, 'best']: 0.0001
[8, 3, 49] vs [8, 9, 'best']: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct2_id_res3-1
[8, 3, 49] vs [8, 3, 53]: 0.0001
/scratch/zz737/fiar/tournaments/tournament_8/checkpoints_mcts100_cpuct0_id_res3-0
[8, 3, 49] vs [8, 3, 49]: 1




In [65]:
players

,tournament,n_res,cp,cpuct,dir_alpha,id
0,12,3,75,2.0,0,0
1,12,9,58,2.0,0,0
2,13,3,75,2.0,0,0
3,13,9,54,2.0,0,0
4,14,3,53,2.0,0,0
5,14,9,44,2.0,0,0
6,15,3,51,2.0,0,0
7,15,9,43,2.0,0,0
8,16,3,55,2.0,0,0
9,8,3,best,2.0,0,0


In [68]:
tour_res.sum(axis=1).sort_values(ascending=False)

9     9.0002
11    7.0006
0     7.0004
2     5.0008
8     5.0008
5     5.0006
4     4.0007
6     3.0006
7     2.0009
1     2.0007
3     1.0008
10   -0.9992
12   -1.9993
dtype: float64

In [66]:
-tour_res.sum(axis=0).sort_values()

4     1.9991
11    1.9991
2     0.9992
3    -2.0005
1    -4.0007
6    -4.0007
10   -4.0009
8    -5.0004
0    -5.0006
9    -6.0005
7    -7.0006
5    -7.0006
12   -8.0005
dtype: object

In [94]:
tour_res

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0001,1,0.0001,1,0.0001,1,1,1,1,1,0.0001,-1,1
1,-1,0.0001,0.0001,0.0001,0.0001,1,0.0001,1,1,-1,0.0001,0.0001,1
2,1,0.0001,0.0001,1,0.0001,0.0001,0.0001,1,0.0001,1,0.0001,0.0001,1
3,0.0001,0.0001,0.0001,-1,-1,1,0.0001,0.0001,0.0001,0.0001,1,0.0001,1
4,0.0001,1,1,0.0001,0.0001,0.0001,-1,1,1,1,0.0001,0.0001,0.0001
5,1,1,1,0.0001,0.0001,0.0001,1,1,-1,0.0001,1,0.0001,0.0001
6,1,1,-1,-1,0.0001,1,0.0001,0.0001,1,0.0001,0.0001,0.0001,1
7,1,0.0001,0.0001,1,-1,0.0001,0.0001,0.0001,0.0001,1,0.0001,0.0001,0.0001
8,1,0.0001,0.0001,0.0001,0.0001,1,0.0001,0.0001,0.0001,1,1,0.0001,1
9,0.0001,1,-1,1,1,1,1,1,1,0.0001,1,1,1


In [97]:
asfirst

array([[0.0001, 1, 0.0001, 1, 0.0001, 1, 1, 1, 1, 1, 0.0001, -2, 1],
       [-2, 0.0001, 0.0001, 0.0001, 0.0001, 1, 0.0001, 1, 1, -2, 0.0001,
        0.0001, 1],
       [1, 0.0001, 0.0001, 1, 0.0001, 0.0001, 0.0001, 1, 0.0001, 1,
        0.0001, 0.0001, 1],
       [0.0001, 0.0001, 0.0001, -2, -2, 1, 0.0001, 0.0001, 0.0001,
        0.0001, 1, 0.0001, 1],
       [0.0001, 1, 1, 0.0001, 0.0001, 0.0001, -2, 1, 1, 1, 0.0001,
        0.0001, 0.0001],
       [1, 1, 1, 0.0001, 0.0001, 0.0001, 1, 1, -2, 0.0001, 1, 0.0001,
        0.0001],
       [1, 1, -2, -2, 0.0001, 1, 0.0001, 0.0001, 1, 0.0001, 0.0001,
        0.0001, 1],
       [1, 0.0001, 0.0001, 1, -2, 0.0001, 0.0001, 0.0001, 0.0001, 1,
        0.0001, 0.0001, 0.0001],
       [1, 0.0001, 0.0001, 0.0001, 0.0001, 1, 0.0001, 0.0001, 0.0001, 1,
        1, 0.0001, 1],
       [0.0001, 1, -2, 1, 1, 1, 1, 1, 1, 0.0001, 1, 1, 1],
       [0.0001, 0.0001, -2, 0.0001, 0.0001, 0.0001, 0.0001, 1, -2, 1,
        0.0001, -2, 0.0001],
       [1, 0.0001, 0.

In [96]:
assecond

array([[0.0001, -1, 2, 0.0001, 0.0001, 2, 2, 2, 2, 0.0001, 0.0001, 2,
        0.0001],
       [2, 0.0001, 0.0001, 0.0001, 2, 2, 2, 0.0001, 0.0001, 2, 0.0001,
        0.0001, -1],
       [0.0001, 0.0001, 0.0001, 0.0001, 2, 2, -1, 0.0001, 0.0001, -1, -1,
        0.0001, 0.0001],
       [2, 0.0001, 2, -1, 0.0001, 0.0001, -1, 2, 0.0001, 2, 0.0001, 2,
        -1],
       [0.0001, 0.0001, 0.0001, -1, 0.0001, 0.0001, 0.0001, -1, 0.0001,
        2, 0.0001, 0.0001, -1],
       [2, 2, 0.0001, 2, 0.0001, 0.0001, 2, 0.0001, 2, 2, 0.0001, 2,
        0.0001],
       [2, 0.0001, 0.0001, 0.0001, -1, 2, 0.0001, 0.0001, 0.0001, 2,
        0.0001, 2, 2],
       [2, 2, 2, 0.0001, 2, 2, 0.0001, 0.0001, 0.0001, 2, 2, 0.0001,
        0.0001],
       [2, 2, 0.0001, 0.0001, 2, -1, 2, 0.0001, 0.0001, 2, -1, 2, 2],
       [2, -1, 2, 0.0001, 2, 0.0001, 0.0001, 2, 2, 0.0001, 2, 2, 0.0001],
       [0.0001, 0.0001, 0.0001, 2, 0.0001, 2, 0.0001, 0.0001, 2, 2,
        0.0001, 0.0001, 0.0001],
       [-1, 0.0001, 0.000

In [92]:
asfirst = copy.copy(tour_res.values)
assecond = copy.copy(tour_res.T.values)

asfirst[asfirst==-1] = asfirst[asfirst==-1]*c
assecond[assecond==1] = assecond[assecond==1]*c

finalscore = asfirst - assecond
finalscore = pd.DataFrame(finalscore,index=tour_res.index,columns=tour_res.columns)

In [82]:
c = 2
np.unique(np.concatenate((tour_res * c - tour_res.T ).apply(pd.unique).values))

array([-3, -2.0001, -1, -0.9998, 0.0001, 1, 1.0002, 1.9999, 3],
      dtype=object)

In [30]:
(tour_res.sum(axis=1)  -tour_res.sum(axis=0)).sort_values(ascending=False)

2      9.9992
1      7.9998
0      6.9997
4      4.0000
8      2.9997
10     2.0002
3      1.0003
5     -1.0001
9     -2.0002
6     -4.0002
11    -7.9994
7     -7.9994
12   -11.9996
dtype: object

In [25]:
tour_res.sum(axis=1).sort_values(ascending=False)

0     10.0003
2     10.0003
1      8.0005
3      7.0006
4      6.0005
9      6.0003
10     5.0006
5      4.0007
8      4.0005
7      3.0006
6      3.0004
11     2.0009
12    -0.9994
dtype: float64

In [26]:
players

,tournament,n_res,cp,cpuct,dir_alpha,id
0,8,3,best,2.0,0,0
1,8,9,best,2.0,0,0
2,12,3,55,2.0,0,0
3,12,9,41,2.0,0,0
4,13,3,55,2.0,0,0
5,13,9,39,2.0,0,0
6,14,3,40,2.0,0,0
7,14,9,28,2.0,0,0
8,15,3,35,2.0,0,0
9,15,9,30,2.0,0,0


In [8]:
for id_1,p_dict_1 in tqdm(players.iterrows()):
    p_dict_1 = dict(p_dict_1)
    val_1,tree_1,ai_1 = get_player(**p_dict_1)
    track_color_1 = p_dict_1['tournament'] not in (8,15,'h')
    for id_2,p_dict_2 in tqdm(players.iterrows()):
        p_dict_2 = dict(p_dict_2)
        val_2,tree_2,ai_2 = get_player(**p_dict_2)
        track_color_2 = p_dict_2['tournament'] not in (8,15,'h')
#         print(f'{p_dict_1} vs {p_dict_2}')

        arena = Arena(ai_1,ai_2, game, display=game.display,tree1=tree_1,tree2=tree_2,track_color=[track_color_1, track_color_2], flip_color=False)
        res=arena.playGame(verbose=False,initboard=None)
#         print(f'{p_dict_1} vs {p_dict_2}: {res}')
        print(f'{list(p_dict_1.values())[:3]} vs {list(p_dict_2.values())[:3]}: {res}')
        tour_res.loc[id_1,id_2] = res
        tour_res.to_pickle('t8_cpuct_diralpha_tournament.p')
        K.clear_session()


[8, 3, 'best'] vs [8, 3, 'best']: 0.0001
[8, 3, 'best'] vs [8, 9, 'best']: 1
[8, 3, 'best'] vs [12, 3, 41]: 1
[8, 3, 'best'] vs [12, 9, 35]: 1
[8, 3, 'best'] vs [13, 3, 35]: 1
[8, 3, 'best'] vs [13, 9, 29]: 1
[8, 3, 'best'] vs [14, 3, 28]: 1
[8, 3, 'best'] vs [14, 9, 20]: 0.0001
[8, 3, 'best'] vs [15, 3, 25]: 1
[8, 3, 'best'] vs [15, 9, 21]: 1



[8, 9, 'best'] vs [8, 3, 'best']: 1
[8, 9, 'best'] vs [8, 9, 'best']: 0.0001
[8, 9, 'best'] vs [12, 3, 41]: 1
[8, 9, 'best'] vs [12, 9, 35]: 0.0001
[8, 9, 'best'] vs [13, 3, 35]: 0.0001
[8, 9, 'best'] vs [13, 9, 29]: 0.0001
[8, 9, 'best'] vs [14, 3, 28]: 0.0001
[8, 9, 'best'] vs [14, 9, 20]: 1
[8, 9, 'best'] vs [15, 3, 25]: 1
[8, 9, 'best'] vs [15, 9, 21]: 1



[12, 3, 41] vs [8, 3, 'best']: 0.0001
[12, 3, 41] vs [8, 9, 'best']: 0.0001
[12, 3, 41] vs [12, 3, 41]: 0.0001
[12, 3, 41] vs [12, 9, 35]: 1
[12, 3, 41] vs [13, 3, 35]: 1
[12, 3, 41] vs [13, 9, 29]: 1
[12, 3, 41] vs [14, 3, 28]: 1
[12, 3, 41] vs [14, 9, 20]: 1
[12, 3, 41] vs [15, 3, 25]: 1
[12, 3, 41] vs [15, 9, 21]: 1



[12, 9, 35] vs [8, 3, 'best']: 0.0001
[12, 9, 35] vs [8, 9, 'best']: 1
[12, 9, 35] vs [12, 3, 41]: 0.0001
[12, 9, 35] vs [12, 9, 35]: 0.0001
[12, 9, 35] vs [13, 3, 35]: 1
[12, 9, 35] vs [13, 9, 29]: 1
[12, 9, 35] vs [14, 3, 28]: 0.0001
[12, 9, 35] vs [14, 9, 20]: -1
[12, 9, 35] vs [15, 3, 25]: 1
[12, 9, 35] vs [15, 9, 21]: -1



[13, 3, 35] vs [8, 3, 'best']: 0.0001
[13, 3, 35] vs [8, 9, 'best']: 0.0001
[13, 3, 35] vs [12, 3, 41]: 1
[13, 3, 35] vs [12, 9, 35]: 0.0001
[13, 3, 35] vs [13, 3, 35]: 1
[13, 3, 35] vs [13, 9, 29]: 1
[13, 3, 35] vs [14, 3, 28]: 1
[13, 3, 35] vs [14, 9, 20]: 0.0001
[13, 3, 35] vs [15, 3, 25]: 1
[13, 3, 35] vs [15, 9, 21]: 1



[13, 9, 29] vs [8, 3, 'best']: -1
[13, 9, 29] vs [8, 9, 'best']: 1
[13, 9, 29] vs [12, 3, 41]: 0.0001
[13, 9, 29] vs [12, 9, 35]: 1
[13, 9, 29] vs [13, 3, 35]: 0.0001
[13, 9, 29] vs [13, 9, 29]: 0.0001
[13, 9, 29] vs [14, 3, 28]: 0.0001
[13, 9, 29] vs [14, 9, 20]: 1
[13, 9, 29] vs [15, 3, 25]: 0.0001
[13, 9, 29] vs [15, 9, 21]: 1



[14, 3, 28] vs [8, 3, 'best']: -1
[14, 3, 28] vs [8, 9, 'best']: 0.0001
[14, 3, 28] vs [12, 3, 41]: 0.0001
[14, 3, 28] vs [12, 9, 35]: 1
[14, 3, 28] vs [13, 3, 35]: -1
[14, 3, 28] vs [13, 9, 29]: 1
[14, 3, 28] vs [14, 3, 28]: 0.0001
[14, 3, 28] vs [14, 9, 20]: 0.0001
[14, 3, 28] vs [15, 3, 25]: 1
[14, 3, 28] vs [15, 9, 21]: 1



[14, 9, 20] vs [8, 3, 'best']: 0.0001
[14, 9, 20] vs [8, 9, 'best']: 1
[14, 9, 20] vs [12, 3, 41]: 1
[14, 9, 20] vs [12, 9, 35]: 1
[14, 9, 20] vs [13, 3, 35]: -1
[14, 9, 20] vs [13, 9, 29]: 0.0001
[14, 9, 20] vs [14, 3, 28]: -1
[14, 9, 20] vs [14, 9, 20]: 0.0001
[14, 9, 20] vs [15, 3, 25]: 1
[14, 9, 20] vs [15, 9, 21]: 0.0001



[15, 3, 25] vs [8, 3, 'best']: 0.0001
[15, 3, 25] vs [8, 9, 'best']: 1
[15, 3, 25] vs [12, 3, 41]: 1
[15, 3, 25] vs [12, 9, 35]: 1
[15, 3, 25] vs [13, 3, 35]: 0.0001
[15, 3, 25] vs [13, 9, 29]: 0.0001
[15, 3, 25] vs [14, 3, 28]: 1
[15, 3, 25] vs [14, 9, 20]: 0.0001
[15, 3, 25] vs [15, 3, 25]: 1
[15, 3, 25] vs [15, 9, 21]: 0.0001



[15, 9, 21] vs [8, 3, 'best']: 1
[15, 9, 21] vs [8, 9, 'best']: 0.0001
[15, 9, 21] vs [12, 3, 41]: 0.0001
[15, 9, 21] vs [12, 9, 35]: 1
[15, 9, 21] vs [13, 3, 35]: 0.0001
[15, 9, 21] vs [13, 9, 29]: 1
[15, 9, 21] vs [14, 3, 28]: -1
[15, 9, 21] vs [14, 9, 20]: -1
[15, 9, 21] vs [15, 3, 25]: 0.0001
[15, 9, 21] vs [15, 9, 21]: 1




In [61]:
i = 0
j = 1
agent_1 = agents[i]
agent_2 = agents[j]
track_color_1 = agent_1[0] not in (8,15,'h')
track_color_2 = agent_2[0] not in (8,15,'h')
get_agent_str = lambda agent, track_color:f'tournament{agent[0]} n_res{agent[1]} iter {agent[2]} color {track_color}'

print(get_agent_str(agent_1,track_color_1),'-vs-',get_agent_str(agent_2,track_color_2))
try:
    mcts_d[agent_1].refresh()
except:
    pass
try:
    mcts_d[agent_2].refresh()
except:
    pass
arena = Arena(ai_d[agent_1],ai_d[agent_2], game, display=game.display,tree1=mcts_d[agent_1],tree2=mcts_d[agent_2],track_color=[track_color_1, track_color_2], flip_color=False)

arena.playGame(verbose=True,initboard=None)



tournament8 n_res3 iter best color False -vs- tournament8 n_res9 iter best color False
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - X - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - X - - - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - O - - - - |
1 |- - - X - X - - - |
2 |- - - - - - - - - |
3 |- - - - O - - - - |
-----------------------
Turn  6 Pla

0.0001

#### 

In [33]:
def diagnostic(b_str,nnet, mcts, run=True,track_color=True, flip_color=False):
    
    if isinstance(b_str,str):
        b = game.get_board_from_xo_str(b_str)
    else:
        b=b_str
    
    if b.sum()!=0:
        b=-b
    s = game.stringRepresentation(b)
    if run:
        mcts.refresh()
        if track_color:
            color = int(b.sum()==0)
            if flip_color:
                color = np.abs(1-color)

#             po = nnet.predict(b,color)[0].reshape(4,9)
            actprob = mcts.getActionProb(b,color).reshape(4,9)
            
            
        else:
#             po = nnet.predict(b)[0].reshape(4,9)
            actprob = mcts.getActionProb(b).reshape(4,9)
    else:
        actprob=None
    self=mcts
    po = mcts.Ps[s].reshape(4,9)
    vals = np.array([self.Qsa[(s, a)] if (s, a) in self.Qsa else -1000 for a in range(self.game.getActionSize())]).astype(np.float32).reshape(4,9)
    counts = np.array([self.Nsa[(s, a)] if (s, a) in self.Nsa else 0 for a in range(self.game.getActionSize())]).reshape(4,9)
    print(b)
    print('policy',po)
    print('act prob',actprob)
    print('counts', counts)
    print('vals', vals)
    
    return po, actprob, counts, vals


In [78]:
b = game.get_board_from_xo_str(b_str)
if b.sum()!=0:
    b=-b
b[3,1] = 1
b=-b
b[3,2]=1
b=-b
diagnostic(-b,nnet_d[agent_2],mcts_d[agent_2],run=False)

[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  1 -1 -1 -1  0  0  0  0]]
policy [[0.011 0.016 0.04  0.01  0.021 0.025 0.031 0.031 0.018]
 [0.028 0.023 0.    0.108 0.041 0.031 0.029 0.02  0.018]
 [0.029 0.031 0.049 0.011 0.027 0.019 0.066 0.027 0.018]
 [0.019 0.    0.    0.    0.    0.133 0.018 0.023 0.03 ]]
act prob None
counts [[0 0 1 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 2 0 0 0]]
vals [[-1000.    -1000.       -0.997 -1000.    -1000.    -1000.       -1.
  -1000.    -1000.   ]
 [-1000.    -1000.    -1000.       -1.       -0.663 -1000.    -1000.
  -1000.    -1000.   ]
 [-1000.    -1000.       -0.974 -1000.    -1000.    -1000.       -1.
  -1000.    -1000.   ]
 [-1000.    -1000.    -1000.    -1000.    -1000.       -0.314 -1000.
  -1000.    -1000.   ]]


(array([[0.011, 0.016, 0.04 , 0.01 , 0.021, 0.025, 0.031, 0.031, 0.018],
        [0.028, 0.023, 0.   , 0.108, 0.041, 0.031, 0.029, 0.02 , 0.018],
        [0.029, 0.031, 0.049, 0.011, 0.027, 0.019, 0.066, 0.027, 0.018],
        [0.019, 0.   , 0.   , 0.   , 0.   , 0.133, 0.018, 0.023, 0.03 ]]),
 None,
 array([[0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0]]),
 array([[-1000.   , -1000.   ,    -0.997, -1000.   , -1000.   , -1000.   ,
            -1.   , -1000.   , -1000.   ],
        [-1000.   , -1000.   , -1000.   ,    -1.   ,    -0.663, -1000.   ,
         -1000.   , -1000.   , -1000.   ],
        [-1000.   , -1000.   ,    -0.974, -1000.   , -1000.   , -1000.   ,
            -1.   , -1000.   , -1000.   ],
        [-1000.   , -1000.   , -1000.   , -1000.   , -1000.   ,    -0.314,
         -1000.   , -1000.   , -1000.   ]], dtype=float32))

In [34]:
b_str='''0 |- - - - O - - - - |
1 |- - - O - - - - - |
2 |- - - - - X - - - |
3 |- - - - - - - - - |'''

diagnostic(b_str, nnet_d[agent_2], mcts_d[agent_2], track_color=track_color_2)

[[ 0  0  0  0 -1  0  0  0  0]
 [ 0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]
policy [[0.003 0.002 0.004 0.009 0.    0.121 0.005 0.002 0.006]
 [0.002 0.002 0.006 0.    0.011 0.011 0.004 0.002 0.002]
 [0.002 0.002 0.341 0.003 0.331 0.    0.089 0.003 0.001]
 [0.005 0.007 0.002 0.006 0.005 0.004 0.002 0.004 0.002]]
act prob [[0.01  0.01  0.01  0.01  0.    0.03  0.01  0.01  0.01 ]
 [0.01  0.01  0.01  0.    0.03  0.02  0.01  0.01  0.01 ]
 [0.01  0.01  0.091 0.061 0.364 0.    0.03  0.01  0.01 ]
 [0.01  0.01  0.01  0.081 0.04  0.02  0.01  0.01  0.01 ]]
counts [[ 1  1  1  1  0  3  1  1  1]
 [ 1  1  1  0  3  2  1  1  1]
 [ 1  1  9  6 36  0  3  1  1]
 [ 1  1  1  8  4  2  1  1  1]]
vals [[   -0.917    -0.748    -0.684    -0.326 -1000.       -0.576    -0.69
     -0.778    -0.767]
 [   -0.903    -0.701    -0.568 -1000.       -0.241    -0.281    -0.457
     -0.837    -0.87 ]
 [   -0.957    -0.802    -0.543    -0.233    -0.291 -1000.       -0.52
     -0.838 

(array([[0.003, 0.002, 0.004, 0.009, 0.   , 0.121, 0.005, 0.002, 0.006],
        [0.002, 0.002, 0.006, 0.   , 0.011, 0.011, 0.004, 0.002, 0.002],
        [0.002, 0.002, 0.341, 0.003, 0.331, 0.   , 0.089, 0.003, 0.001],
        [0.005, 0.007, 0.002, 0.006, 0.005, 0.004, 0.002, 0.004, 0.002]]),
 array([[0.01 , 0.01 , 0.01 , 0.01 , 0.   , 0.03 , 0.01 , 0.01 , 0.01 ],
        [0.01 , 0.01 , 0.01 , 0.   , 0.03 , 0.02 , 0.01 , 0.01 , 0.01 ],
        [0.01 , 0.01 , 0.091, 0.061, 0.364, 0.   , 0.03 , 0.01 , 0.01 ],
        [0.01 , 0.01 , 0.01 , 0.081, 0.04 , 0.02 , 0.01 , 0.01 , 0.01 ]]),
 array([[ 1,  1,  1,  1,  0,  3,  1,  1,  1],
        [ 1,  1,  1,  0,  3,  2,  1,  1,  1],
        [ 1,  1,  9,  6, 36,  0,  3,  1,  1],
        [ 1,  1,  1,  8,  4,  2,  1,  1,  1]]),
 array([[   -0.917,    -0.748,    -0.684,    -0.326, -1000.   ,    -0.576,
            -0.69 ,    -0.778,    -0.767],
        [   -0.903,    -0.701,    -0.568, -1000.   ,    -0.241,    -0.281,
            -0.457,    -0.837,   

In [117]:
tournament_dir = 'tournament_8'
# other_type = f'n_res={n_res};color=False'
better_than_last = True
color = False

In [102]:
COLUMNS = cd.COLUMNS
root_dir = cd.root_dir

In [112]:
subdir_split

['checkpoints', 'mcts100', 'cpuct5e-01', 'id', 'res3-0']

In [125]:
kwargs = {'color':False}

In [126]:
if 'color' in kwargs.keys():
    color = kwargs['color']
else:
    color = None

all_players = pd.DataFrame([],columns=COLUMNS)

d = tournament_dir
# for d in tournaments_dirs:
tour_dir = os.path.join(root_dir,d)
tournament = int(d.split('_')[1])
subdirs = os.listdir(tour_dir)
for subdir in subdirs:
    if subdir.startswith('checkpoints_'):
        subdir_split = subdir.split('_')
        mctsN = subdir_split[1]
        cpuctX = subdir_split[2]

        mctsN = int(mctsN[4:])
        cpuctX = float(cpuctX[5:]) #careful about consequence (int -> float) for loading?

        # if len(subdir_split) > 3: # then it should have id-xxx or id_x, should make it consistent
        if len(subdir_split)==5: # which means the folder name contains id_x; but we want id-x 
            id = ';'.join([d,*subdir_split[1:-1]])
            id += f'-{subdir_split[-1]}'
        else:
            id = ';'.join([d,*subdir_split[1:]]) # combine tournament_x and mctsN_cpuctX_idOOO, to create an identifier for the trained nn+mcts
            
        other_type = ''
        if 'res' in subdir_split[-1]:
            other_type +=f"res={subdir_split[-1][3:].split('-')[0]};"
        if color is not None:
            other_type += f'color={color}'
        
        location = os.path.join(tour_dir,subdir) # e.g. /scratch/zz737/fiar/tournaments/tournament_1/checkpoints_mcts100_cpuct1
        files = os.listdir(location)
        for f in files:
            if f.endswith('index') and (not f.startswith('temp')): # only the succeeded models are saved, with .index, in tournament1 and 5, not in 4
                if f.startswith('best'):
                    value_func_iter = mcts_iter = 'best' # = best
                    curr_id = ';'.join([id,value_func_iter]) # add iter info to id
                elif f.startswith('checkpoint_'):
                    iter =f.split('.')[0].split('_')[1]
                    value_func_iter = mcts_iter = int(iter) # = number
                    curr_id = ';'.join([id,iter]) # add iter info to id

                fn_without_index = '.'.join(f.split('.')[:-1])
                mcts_location = value_func_location = os.path.join(location,fn_without_index)

                player_info={'tree_type':'mcts', 'value_func_type':'nn', 'n_mcts':mctsN,'cpuct':cpuctX, \
                'value_func_iter':value_func_iter,'mcts_iter':mcts_iter,'id':curr_id,'tournament':tournament,'mcts_location':mcts_location,'value_func_location':value_func_location}
                
                if other_type != '':
                    player_info['other_type'] = other_type
                    
                
                player_info = pd.Series(player_info)
                all_players = all_players.append(player_info,ignore_index=True)


# for tournament_1, and all the best iter, we are sure they are better than last; tournament_4 not sure:
all_players['better_than_last'] = better_than_last
# all_players.loc[all_players['tournament']==1,'better_than_last'] = True
all_players.loc[all_players['value_func_iter']=='best','better_than_last'] = True

# sort df by tournament, n_mcts, cpuct, then iter
gpb = all_players.groupby(['tournament','n_mcts','cpuct'])
sort_fun = lambda x:x['value_func_iter'].replace('best',pd.NA).sort_values()
gpb_res = gpb.apply(sort_fun)
if gpb_res.shape[0]==1: # if only one sub category then it would be a df with one row, tournament, mcts, cpuct as the index, multiple columns for the index of the sorted value
    inds = gpb_res.columns
else:
    inds=gpb_res.index.get_level_values(-1) # return multiindex, only need the last one to rearrange the df

all_players= all_players.loc[inds]
all_players = all_players.loc[~all_players['id'].duplicated()] # there might be duplicated rows, not sure why, need to be dropped
all_players = all_players.reset_index(drop=True)

In [127]:
all_players

,tree_type,value_func_type,other_type,n_mcts,cpuct,value_func_iter,mcts_iter,n_bfts,pruning_thresh,center,2_con,2_uncon,3,4,oppo_scale,id,tournament,mcts_location,value_func_location,better_than_last
0,mcts,nn,res=3;color=False,100,0.5,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct5e-01;id-res3-0;1,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
1,mcts,nn,res=3;color=False,100,0.5,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct5e-01;id-res3-0;3,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
2,mcts,nn,res=3;color=False,100,0.5,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct5e-01;id-res3-0;4,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
3,mcts,nn,res=3;color=False,100,0.5,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct5e-01;id-res3-0;6,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
4,mcts,nn,res=3;color=False,100,0.5,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct5e-01;id-res3-0;7,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,mcts,nn,res=9;color=False,100,2.0,53,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct2;id-res9-0;53,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
113,mcts,nn,res=9;color=False,100,2.0,57,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct2;id-res9-0;57,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
114,mcts,nn,res=9;color=False,100,2.0,best,best,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct2;id-res9-0;best,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True
115,mcts,nn,res=3;color=False,100,2.0,best,best,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tournament_8;mcts100;cpuct2;id-res3-0;best,8,/scratch/zz737/fiar/tournaments/tournament_8/c...,/scratch/zz737/fiar/tournaments/tournament_8/c...,True


In [105]:
int(float('2e+00'))

2